### Library Import

In [1]:
import math
import numpy as np
import pandas as pd
import hashlib as hl

### Data Import

In [2]:
df = pd.read_csv('exam2.csv')
df = df.loc[df['hash']==hl.md5('pcg180000'.encode('utf-8')).hexdigest()]
df = df.iloc[:,1:].T
# Assuming unscaled dataset as z
df.columns = ['z']
df.describe()

,z
count,200.000000
mean,-8.725276
std,1.011065
min,-9.524298
25%,-9.385453
50%,-8.970802
75%,-8.373185
max,-3.680400


### Estimation

In [3]:
# Taking min and max of z as initial lower and upper bounds
z = df.values
zlb = z.min()
zub = z.max()

for i in range(10):
    # Scaling z to x
    x = (z-zlb)/(zub-zlb)
    x = x[x>0]
    x = x[x<1]
    n = x.shape[0]
    x = np.reshape(x,(n,1))

    a = 1 #Assumption
    for i in range(100):
        db = np.log(1-(x**a))
        b  = -n/db.sum()
        da = np.log(x)*(1-(np.multiply(x**a,b)))/(1-(x**a))
        a  = -n/da.sum()

    # Order statistics to get new bounds for scaled distribution
    x1 = n*a*b*(np.power(x.min(),a))*(np.power(1-np.power(x.min(),a),(b*n)-1))
    xn = n*a*b*(np.power(x.max(),a))*(np.power(1-np.power(x.max(),a),b-1))*(np.power(1-np.power(1-x.max(),b),n-1))

    # Solving lower and upper bounds for unscaled z distribution
    p = [[1-x1,x1],[1-xn,xn]]
    q = [[z.min()],[z.max()]]
    zlb = np.linalg.solve(p,q)[0]
    zub = np.linalg.solve(p,q)[1]

### Parameters and Bounds

In [4]:
print('Parameter a =',a,'and Parameter b =',b)
print('Lower Bound =',zlb,'and Upper Bound =',zub)

Parameter a = 0.6989828379084404 and Parameter b = 1.1665075539647058
Lower Bound = [-9.524332] and Upper Bound = [-9.30711047]
